# Throw a neural net at a crap ton of indicators. Try to predict if the stock will go up or down in the next 5 days.

In [1]:
import numpy as np
import pandas as pd
import yfinance
import matplotlib.pyplot as plt
import seaborn as sns
import indicators
%matplotlib inline
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense,Dropout
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import classification_report,confusion_matrix

In [15]:
def get_data(ticker):

    # Reading in stock data 
    data = yfinance.download(ticker)
    print(data.shape)
    data['Date'] = data.index
    data.reset_index(drop=True,inplace=True)
    data.head()

    for days in [5,7,14,21,50,100]:
        data = indicators.accumulation_distribution(data,days)
        data = indicators.average_true_range(data,days)
        data = indicators.bollinger_bands(data,days)
        data = indicators.commodity_channel_index(data,days)
        data = indicators.coppock_curve(data,days)
        #data = indicators.donchian_channel(data,days)
        data = indicators.ease_of_movement(data,days)
        data = indicators.exponential_moving_average(data,days)
        data = indicators.force_index(data,days)
        data = indicators.keltner_channel(data,days)
        data = indicators.macd(data,days,days)
        data = indicators.momentum(data,days)
        data = indicators.money_flow_index(data,days)
        data = indicators.moving_average(data,days)
        data = indicators.on_balance_volume(data,days)
        data = indicators.rate_of_change(data,days)
        data = indicators.relative_strength_index(data,days)
        data = indicators.standard_deviation(data,days)
        data = indicators.stochastic_oscillator_d(data,days)
        data = indicators.trix(data,days)
        data = indicators.vortex_indicator(data,days)



    data = indicators.mass_index(data)
    data = indicators.ppsr(data)
    data = indicators.stochastic_oscillator_k(data)
    data = indicators.ultimate_oscillator(data)    
    data = indicators.chaikin_oscillator(data)

    data = data.replace([np.inf, -np.inf], np.nan)
    data = data.dropna()
    
    return data

In [16]:
def model(ticker):
    data = get_data(ticker)
    copy_data = data.copy()
    data = data[:-20] #Drop last 20 days of data, save this for testing.
    
    
    # Calculating next n day price
    data['Close5'] = data['Close'].shift(-5)

    # Calculating next n direction
    data['Direction5'] = data['Close5'] > data['Close']
    data = data.dropna()
    data.tail(10)



    # Creating train and test set
    X,y = data.drop(['Close5','Direction5','Date'],axis=1),data['Direction5']

    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_inputs = scaler.fit_transform(X)
    scaled_inputs = pd.DataFrame(scaled_inputs,columns=X.columns)

    X_train,X_test,y_train,y_test = train_test_split(scaled_inputs,y,test_size=.03,random_state=101)



    # Network time
    model = Sequential()
    model.add(Dense(100, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))


    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(X_train, y_train,validation_data=(X_test,y_test), epochs=300, batch_size=100, verbose=0)


    # Calculating next n day price
    copy_data['Close5'] = copy_data['Close'].shift(-5)



    # Calculating next n direction
    copy_data['Direction5'] = copy_data['Close5'] > copy_data['Close']
    copy_data = copy_data.dropna()

    ts = pd.Series(copy_data['Direction5'].values)

    X = copy_data.drop(['Close5','Direction5','Date'],axis=1, inplace=True)#,copy_data['Direction5']

    rows, columns = copy_data.shape
    copy_data = copy_data[rows-15:rows]
    ts = ts[rows-15:rows]

    preds = model.predict(copy_data)
    preds = [i > .50 for i in preds]
    predicted = []
    for pred in preds:
        val = pred[0]
        predicted.append(val)
    print(classification_report(ts,predicted))
    print(confusion_matrix(ts,predicted))


In [17]:
#stocks chosen with : https://raybb.github.io/random-stock-picker/
stocks = ['AAPL', 'MUH', 'GLU', 'D', 'SAVA']
for stock in stocks:
    model(stock)

[*********************100%***********************]  1 of 1 completed
(9940, 6)
              precision    recall  f1-score   support

       False       0.00      0.00      0.00         1
        True       0.80      0.29      0.42        14

    accuracy                           0.27        15
   macro avg       0.40      0.14      0.21        15
weighted avg       0.75      0.27      0.39        15

[[ 0  1]
 [10  4]]
[*********************100%***********************]  1 of 1 completed
(5589, 6)


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

       False       0.00      0.00      0.00         7
        True       0.53      1.00      0.70         8

    accuracy                           0.53        15
   macro avg       0.27      0.50      0.35        15
weighted avg       0.28      0.53      0.37        15

[[0 7]
 [0 8]]
[*********************100%***********************]  1 of 1 completed
(4021, 6)


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

       False       0.00      0.00      0.00         6
        True       0.60      1.00      0.75         9

    accuracy                           0.60        15
   macro avg       0.30      0.50      0.37        15
weighted avg       0.36      0.60      0.45        15

[[0 6]
 [0 9]]
[*********************100%***********************]  1 of 1 completed
(10129, 6)
              precision    recall  f1-score   support

       False       0.45      0.71      0.56         7
        True       0.50      0.25      0.33         8

    accuracy                           0.47        15
   macro avg       0.48      0.48      0.44        15
weighted avg       0.48      0.47      0.44        15

[[5 2]
 [6 2]]
[*********************100%***********************]  1 of 1 completed
(4991, 6)
              precision    recall  f1-score   support

       False       0.33      1.00      0.50         3
        True       1.00      0.50      0.67     

### Conclusion: This doesn't seem to work...